In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH02-Prompt-Practice")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH02-Prompt-Practice


In [4]:
from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import stream_response

# 객체 생성
llm = ChatOpenAI(
    temperature=0,  # 창의성
    model_name="gpt-4.1-nano",  # 모델명
)

In [6]:
# ==== 설치(최초 1회) ====
# pip install -U langchain langchain-openai langchain-community faiss-cpu

import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import FAISS


# =========================
# 1) 공통: 예시(examples)
# =========================
# 형식을 일부러 일정하게 맞춰 예시-스타일을 학습시키도록 함
examples = [
    {
        "question": "한국의 대표 음식은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 한국의 전통 음식에는 어떤 것들이 있나요?
  중간 답변: 김치, 불고기, 비빔밥, 떡볶이 등이 있습니다.
  추가 질문: 이 중 가장 대표적인 음식은 무엇인가요?
  중간 답변: 김치가 가장 대표적인 한국 음식입니다.
  최종 답변은: 김치
  """,
    },
    {
        "question": "일본의 전통 음식은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 일본의 대표적인 요리에는 어떤 것들이 있나요?
  중간 답변: 스시, 라멘, 우동, 덴푸라 등이 있습니다.
  추가 질문: 이 중 가장 유명한 전통 음식은 무엇인가요?
  중간 답변: 스시가 가장 유명한 일본 전통 음식입니다.
  최종 답변은: 스시
  """,
    },
    {
        "question": "파이썬에서 가장 많이 사용되는 웹 프레임워크는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 파이썬의 웹 프레임워크에는 어떤 것들이 있나요?
  중간 답변: Django, Flask, FastAPI, Pyramid 등이 있습니다.
  추가 질문: 이 중 가장 널리 사용되는 것은 무엇인가요?
  중간 답변: Django가 가장 많이 사용되는 파이썬 웹 프레임워크입니다.
  최종 답변은: Django
  """,
    },
    {
        "question": "자바스크립트의 인기 있는 프론트엔드 프레임워크는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 자바스크립트 프론트엔드 프레임워크에는 어떤 것들이 있나요?
  중간 답변: React, Vue, Angular, Svelte 등이 있습니다.
  추가 질문: 현재 가장 인기 있는 것은 무엇인가요?
  중간 답변: React가 현재 가장 인기 있는 프론트엔드 프레임워크입니다.
  최종 답변은: React
  """,
    },
    {
        "question": "서울에서 가장 높은 산은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 서울에 있는 주요 산들은 무엇인가요?
  중간 답변: 북한산, 남산, 관악산, 도봉산 등이 있습니다.
  추가 질문: 이 중 가장 높은 산은 무엇인가요?
  중간 답변: 북한산이 836.5m로 가장 높습니다.
  최종 답변은: 북한산
  """,
    },
    {
        "question": "부산의 유명한 해수욕장은 어디인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 부산에는 어떤 해수욕장들이 있나요?
  중간 답변: 해운대, 광안리, 송정, 다대포 해수욕장 등이 있습니다.
  추가 질문: 이 중 가장 유명한 곳은 어디인가요?
  중간 답변: 해운대 해수욕장이 가장 유명합니다.
  최종 답변은: 해운대 해수욕장
  """,
    },
]

# 예시가 출력될 포맷
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Q: {question}\nA:\n{answer}\n"
)

# =========================
# 2) 제로샷 체인 (비교용)
# =========================
zero_prompt = ChatPromptTemplate.from_messages([
    ("system", "사용자 질문에 간결하고 정확하게 답하라."),
    ("user", "{question}")
])
zero_chain = zero_prompt | llm | StrOutputParser()

# =========================
# 3) FewShot: 고정 예시 사용
# =========================
fewshot_prompt_fixed = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="=== 예시 시작 ===\n",
    suffix="=== 예시 끝 ===\n사용자 질문: {question}\n최종 답변:",
    input_variables=["question"],
)
fewshot_chain_fixed = fewshot_prompt_fixed | llm | StrOutputParser()

# =========================
# 4) FewShot: SemanticSimilarityExampleSelector 사용
# =========================
# - 입력 질문과 가장 유사한 예시 k개만 뽑아 few-shot에 넣기
selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,
    embeddings=OpenAIEmbeddings(),      # 임베딩 백엔드
    vectorstore_cls=FAISS,              # 벡터스토어
    k=2                                  # 가장 가까운 예시 2개만 사용
)

fewshot_prompt_selected = FewShotPromptTemplate(
    example_selector=selector,          # <- 고정 examples 대신 selector 사용
    example_prompt=example_prompt,
    prefix=(
        "아래는 입력 질문과 유사도가 높은 예시만 선별한 것이다.\n"
        "예시의 형식과 톤을 따라 최종 답만 출력하라.\n\n=== 예시 시작 ===\n"
    ),
    suffix="=== 예시 끝 ===\n사용자 질문: {question}\n최종 답변:",
    input_variables=["question"],
)
fewshot_chain_selected = fewshot_prompt_selected | llm | StrOutputParser()

# =========================
# 5) 실행: 같은 질문으로 3가지 비교
# =========================
question = "서울의 유명한 산은 어디인가요?"

print("▼ [제로샷]")
print(zero_chain.invoke({"question": question}))
print("\n" + "-"*80 + "\n")

print("▼ [FewShot - 고정 예시]")
print(fewshot_chain_fixed.invoke({"question": question}))
print("\n" + "-"*80 + "\n")

print("▼ [FewShot - SemanticSimilarityExampleSelector(k=2)]")
print(fewshot_chain_selected.invoke({"question": question}))


▼ [제로샷]
서울의 유명한 산은 북한산, 관악산, 도봉산, 인왕산 등이 있습니다.

--------------------------------------------------------------------------------

▼ [FewShot - 고정 예시]
이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 서울에 있는 유명한 산들은 무엇인가요?
  중간 답변: 북한산, 남산, 관악산, 도봉산 등이 있습니다.
  추가 질문: 이 중 가장 유명하거나 대표적인 산은 어디인가요?
  중간 답변: 남산이 서울을 대표하는 산으로 유명합니다.
  최종 답변은: 남산

--------------------------------------------------------------------------------

▼ [FewShot - SemanticSimilarityExampleSelector(k=2)]
이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 서울에 있는 유명한 산들은 무엇인가요?
  중간 답변: 북한산, 남산, 관악산, 도봉산 등이 있습니다.
  추가 질문: 이 중 가장 유명하거나 높은 산은 무엇인가요?
  중간 답변: 북한산과 남산이 가장 유명하며, 북한산이 높이도 높습니다.
  최종 답변은: 북한산


In [18]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


examples = [
    {
        "question": "프랑스의 수도는 어디인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 프랑스의 주요 도시는 어디인가요?
  중간 답변: 파리, 리옹, 마르세유, 툴루즈 등이 있습니다.
  추가 질문: 이 중 수도는 어디인가요?
  중간 답변: 수도는 파리입니다.
  최종 답변은: 파리
  """,
    },
    {
        "question": "태양계에서 가장 큰 행성은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 태양계의 거대 가스 행성은 무엇이 있나요?
  중간 답변: 목성, 토성, 천왕성, 해왕성이 있습니다.
  추가 질문: 이 중 크기가 가장 큰 행성은 무엇인가요?
  중간 답변: 목성이 가장 큽니다.
  최종 답변은: 목성
  """,
    },
    {
        "question": "HTTP와 HTTPS의 주요 차이는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: HTTPS에서 보안을 제공하는 기술은 무엇인가요?
  중간 답변: TLS/SSL을 사용해 통신을 암호화합니다.
  추가 질문: 그러면 핵심 차이는 무엇인가요?
  중간 답변: HTTPS는 암호화와 서버 인증을 제공하고 HTTP는 제공하지 않습니다.
  최종 답변은: TLS로 암호화/인증 제공
  """,
    },
    {
        "question": "파이썬 내장 정렬 알고리즘의 기반은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 파이썬의 list.sort()와 sorted()가 사용하는 알고리즘은?
  중간 답변: Timsort를 사용합니다.
  추가 질문: Timsort의 특징은 무엇인가요?
  중간 답변: 병합정렬과 삽입정렬의 하이브리드로 실전 데이터에 강합니다.
  최종 답변은: Timsort
  """,
    },
    {
        "question": "FIFO(선입선출)에 적합한 자료구조는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 선입선출을 구현하는 대표 구조는?
  중간 답변: 큐(Queue)입니다.
  추가 질문: 파이썬에서 큐를 구현할 때 흔히 쓰는 모듈은?
  중간 답변: collections.deque를 자주 사용합니다.
  최종 답변은: 큐(Queue)
  """,
    },
    {
        "question": "가장 널리 쓰이는 관계형 데이터베이스는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 대표적인 RDBMS는 무엇이 있나요?
  중간 답변: MySQL, PostgreSQL, Oracle, SQL Server 등이 있습니다.
  추가 질문: 오픈소스 중 특히 널리 쓰이는 것은?
  중간 답변: MySQL이 매우 널리 쓰입니다.
  최종 답변은: MySQL
  """,
    },
    {
        "question": "Git에서 변경된 파일을 스테이지에 올리는 명령은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 특정 파일을 스테이징하려면?
  중간 답변: git add <파일명>을 사용합니다.
  추가 질문: 모든 변경을 스테이징하려면?
  중간 답변: git add . 를 사용할 수 있습니다.
  최종 답변은: git add
  """,
    },
    {
        "question": "클라우드 서비스 모델 중 Gmail은 어떤 분류에 해당하나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 클라우드 서비스 모델의 종류는?
  중간 답변: IaaS, PaaS, SaaS가 있습니다.
  추가 질문: 완성된 애플리케이션을 사용하는 모델은?
  중간 답변: SaaS입니다.
  최종 답변은: SaaS
  """,
    },
    {
        "question": "행렬 곱셈은 어떤 경우에 주로 사용되나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 대표적인 활용 분야는?
  중간 답변: 컴퓨터 그래픽스의 좌표변환, 머신러닝의 선형변환 등입니다.
  추가 질문: 가장 핵심적인 요점은?
  중간 답변: 선형변환을 조합하고 표현하는 데 사용됩니다.
  최종 답변은: 선형변환/좌표변환
  """,
    },
    {
        "question": "공정한 동전 한 번 던졌을 때 앞면이 나올 확률은 얼마인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 아니오.
  최종 답변은: 0.5
  """,
    },
    {
        "question": "서버에서 많이 사용하는 리눅스 배포판은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 대표 배포판은?
  중간 답변: Ubuntu, CentOS/AlmaLinux, Debian 등이 있습니다.
  추가 질문: 최근 클라우드에서 널리 쓰이는 것은?
  중간 답변: Ubuntu LTS가 널리 쓰입니다.
  최종 답변은: Ubuntu LTS
  """,
    },
    {
        "question": "1차원 레이아웃을 잡을 때 주로 사용하는 CSS 기술은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 현대 CSS 레이아웃의 핵심 기술은?
  중간 답변: Flexbox와 Grid입니다.
  추가 질문: 1차원(가로/세로 한 축) 배치에 적합한 것은?
  중간 답변: Flexbox입니다.
  최종 답변은: Flexbox
  """,
    },
    {
        "question": "딥러닝 은닉층에서 가장 흔히 쓰이는 활성화 함수는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 대표 활성화 함수는?
  중간 답변: ReLU, Sigmoid, Tanh 등이 있습니다.
  추가 질문: 은닉층에서 기본 선택으로 널리 쓰이는 것은?
  중간 답변: ReLU입니다.
  최종 답변은: ReLU
  """,
    },
    {
        "question": "서브워드 토크나이저로 널리 쓰이는 방법은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 대표 기법은?
  중간 답변: BPE, WordPiece, Unigram 등이 있습니다.
  추가 질문: 오픈소스 생태계에서 특히 널리 쓰이는 것은?
  중간 답변: BPE가 매우 널리 쓰입니다.
  최종 답변은: BPE
  """,
    },
    {
        "question": "OSI 7계층은 총 몇 개의 계층으로 구성되나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 아니오.
  최종 답변은: 7계층
  """,
    },
    {
        "question": "SQL에서 두 테이블의 교집합 레코드만 가져오는 조인은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: OUTER JOIN과 대비되는 조인은?
  중간 답변: INNER JOIN입니다.
  추가 질문: 교집합 성격을 가지는 조인은?
  중간 답변: INNER JOIN이 해당합니다.
  최종 답변은: INNER JOIN
  """,
    },
    {
        "question": "파이썬에서 가상환경을 만드는 기본 명령은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 표준 라이브러리로 만드는 방법은?
  중간 답변: python -m venv <이름>을 사용합니다.
  추가 질문: 관례적으로 자주 쓰는 이름은?
  중간 답변: venv 혹은 .venv를 자주 씁니다.
  최종 답변은: python -m venv venv
  """,
    },
    {
        "question": "Pandas로 CSV 파일을 읽는 함수는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 아니오.
  최종 답변은: pandas.read_csv
  """,
    },
    {
        "question": "GPU 연산 플랫폼 CUDA를 만든 회사는 어디인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 아니오.
  최종 답변은: NVIDIA
  """,
    },
    {
        "question": "REST API 설계에서 리소스 조회에 주로 사용하는 HTTP 메서드는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 아니오.
  최종 답변은: GET
  """,
    },
    {
        "question": "이진 탐색(Binary Search)을 사용하기 위한 전제 조건은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 어떤 형태의 입력 배열이어야 하나요?
  중간 답변: 정렬된 배열이어야 합니다.
  추가 질문: 오름차순/내림차순 모두 가능한가요?
  중간 답변: 네, 정렬 기준만 일관되면 가능합니다.
  최종 답변은: 정렬된 배열
  """,
    },
    {
        "question": "빅오 표기법에서 퀵정렬의 평균 시간복잡도는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 대표 정렬들의 평균 시간복잡도는?
  중간 답변: 병합/힙/퀵 모두 평균 O(n log n)입니다.
  추가 질문: 퀵정렬의 평균 시간복잡도는?
  중간 답변: O(n log n)입니다.
  최종 답변은: O(n log n)
  """,
    },
    {
        "question": "파이썬에서 비동기 함수를 정의하는 키워드는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 아니오.
  최종 답변은: async
  """,
    },
    {
        "question": "Docker에서 컨테이너를 백그라운드로 실행하는 옵션은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: docker run에 어떤 옵션을 쓰나요?
  중간 답변: -d 옵션을 사용합니다.
  추가 질문: -d의 의미는?
  중간 답변: detached 모드로 실행합니다.
  최종 답변은: -d (detached)
  """,
    },
    {
        "question": "자바스크립트에서 상수(변경 불가 참조)를 선언하는 키워드는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 아니오.
  최종 답변은: const
  """,
    },
    {
        "question": "삼성회장 이재용이 주로 쓰는 시계 브랜드의 사장의 이름은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: (1) '주로 쓰는'의 근거가 되는 공신력 있는 출처가 있나요? (2) 특정 시점(연/월)을 명시해 주시겠어요?
  중간 답변: 개인의 사적 소비 습관은 공개·검증이 어려우며, 브랜드 CEO/사장은 시점에 따라 변동됩니다. 근거 출처와 기준 시점이 필요합니다.
  추가 질문: 공개 확인 가능한 근거(공식 인터뷰/기사/사진 등)를 제공하면 교차검증이 가능합니다. 제공 가능하신가요?
  중간 답변: 현재로서는 공신력 있는 증거가 없어 단정할 수 없습니다.
  최종 답변은: 공신력 있는 출처와 시점이 없이는 확답 불가(사생활 영역). 확인 가능한 자료와 기준 시점을 제공해 주세요.
  """,
    },
    {
        "question": "롤렉스(ROLEX)의 CEO는 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 기준 시점(예: 2025-08 기준)을 명시해 주시겠어요?
  중간 답변: CEO는 시점에 따라 변동될 수 있으므로, 날짜를 특정해야 정확합니다.
  추가 질문: 또한 공식 웹사이트나 최근 보도자료 등 신뢰 가능한 출처 확인이 필요합니다.
  중간 답변: 시점과 출처를 확인하면 정확한 답을 제시할 수 있습니다.
  최종 답변은: 기준 시점과 공식 출처 확인이 필요합니다(시점 미명시 상태에서는 확답 보류).
  """,
    },
    {
        "question": "이재용의 시계 한 벌 가격은 얼마인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: (1) 특정 모델/사진/행사 등 확인 가능한 근거가 있나요? (2) 어느 시점의 거래가(정가/중고/한정판)인가요?
  중간 답변: 개인의 사적 소비 금액은 비공개인 경우가 많고, 모델·시점·상태에 따라 가격 변동폭이 큽니다.
  추가 질문: 공신력 있는 출처 링크를 제공해 주시겠어요?
  중간 답변: 현재로선 검증 불가입니다.
  최종 답변은: 사생활 영역 및 검증 불가 항목으로 확답 불가. 모델·시점·출처 제공 시 범위 추정은 가능.
  """,
    },
    {
        "question": "네이버의 CEO는 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 어느 시점 기준인가요? (연/월)
  중간 답변: 경영진은 수시로 변경될 수 있어 날짜를 특정해야 정확합니다.
  추가 질문: 최근 공시/보도자료/IR 페이지 등 공식 출처 확인이 가능한가요?
  중간 답변: 시점과 출처를 확인하면 정확한 정보를 제공할 수 있습니다.
  최종 답변은: 기준 시점과 공식 출처 확인이 필요합니다(시점 미명시 시 확답 보류).
  """,
    },
    {
        "question": "최신 아이폰의 출고가는 얼마인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: (1) 모델명(예: Pro/Pro Max/용량) (2) 지역/통신(한국/미국, 자급제/약정) (3) 기준 시점은?
  중간 답변: 모델·용량·지역·환율·프로모션에 따라 가격이 크게 달라집니다.
  추가 질문: 어떤 조합으로 조회할까요?
  중간 답변: 조건 확정 시 정확한 가격 범위를 제시할 수 있습니다.
  최종 답변은: 모델/지역/시점을 명시해 주세요. 조건 없이는 단일 금액 제시 불가.
  """,
    },
    {
        "question": "유명인의 자택 주소는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 아니오.
  최종 답변은: 개인정보 및 안전을 침해하는 요청으로 제공 불가(사생활 보호 원칙).
  """,
    },
    {
        "question": "올해 UEFA 챔피언스리그 우승팀은 어디인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: '올해'의 기준 연도를 명시해 주세요(예: 2024/2025).
  중간 답변: 시즌은 보통 연도跨越이며 결승 시점에 따라 결과가 다릅니다.
  추가 질문: 시즌(YYYY–YY)을 알려 주시면 정확히 답변할 수 있습니다.
  중간 답변: 시즌 명시가 필요합니다.
  최종 답변은: 시즌/연도를 명시해 주세요. 그에 맞춰 우승팀을 알려 드리겠습니다.
  """,
    },
    {
        "question": "테슬라의 최신 분기 실적 매출은 얼마인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 어느 회계분기(예: 2025 Q1)와 통화 단위(USD/원)인가요?
  중간 답변: 분기/환율/보고 기준에 따라 수치가 달라집니다.
  추가 질문: 공식 10-Q/보도자료 등 출처 범위를 지정해 주시겠어요?
  중간 답변: 분기·출처 확정 시 정확한 수치를 제시 가능합니다.
  최종 답변은: 분기·통화·출처를 명시해 주세요. 조건 없이는 '최신' 정의가 모호하여 확답 보류.
  """,
    },
    {
        "question": "한국의 대표 음식은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 한국의 전통 음식에는 어떤 것들이 있나요?
중간 답변: 김치, 불고기, 비빔밥, 떡볶이 등이 있습니다.
추가 질문: 이 중 가장 대표적인 음식은 무엇인가요?
중간 답변: 김치가 가장 대표적인 한국 음식입니다.
최종 답변은: 김치
""",
    },
    {
        "question": "일본의 전통 음식은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 일본의 대표적인 요리에는 어떤 것들이 있나요?
중간 답변: 스시, 라멘, 우동, 덴푸라 등이 있습니다.
추가 질문: 이 중 가장 유명한 전통 음식은 무엇인가요?
중간 답변: 스시가 가장 유명한 일본 전통 음식입니다.
최종 답변은: 스시
""",
    },
    {
        "question": "파이썬에서 가장 많이 사용되는 웹 프레임워크는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 파이썬의 웹 프레임워크에는 어떤 것들이 있나요?
중간 답변: Django, Flask, FastAPI, Pyramid 등이 있습니다.
추가 질문: 이 중 가장 널리 사용되는 것은 무엇인가요?
중간 답변: Django가 가장 많이 사용되는 파이썬 웹 프레임워크입니다.
최종 답변은: Django
""",
    },
    {
        "question": "자바스크립트의 인기 있는 프론트엔드 프레임워크는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 자바스크립트 프론트엔드 프레임워크에는 어떤 것들이 있나요?
중간 답변: React, Vue, Angular, Svelte 등이 있습니다.
추가 질문: 현재 가장 인기 있는 것은 무엇인가요?
중간 답변: React가 현재 가장 인기 있는 프론트엔드 프레임워크입니다.
최종 답변은: React
""",
    },
    {
        "question": "서울에서 가장 높은 산은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 서울에 있는 주요 산들은 무엇인가요?
중간 답변: 북한산, 남산, 관악산, 도봉산 등이 있습니다.
추가 질문: 이 중 가장 높은 산은 무엇인가요?
중간 답변: 북한산이 836.5m로 가장 높습니다.
최종 답변은: 북한산
""",
    },
    {
        "question": "부산의 유명한 해수욕장은 어디인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 부산에는 어떤 해수욕장들이 있나요?
중간 답변: 해운대, 광안리, 송정, 다대포 해수욕장 등이 있습니다.
추가 질문: 이 중 가장 유명한 곳은 어디인가요?
중간 답변: 해운대 해수욕장이 가장 유명합니다.
최종 답변은: 해운대 해수욕장
""",
    },
    {
        "question": "프랑스의 수도는 어디인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 프랑스의 주요 도시에는 어떤 곳들이 있나요?
중간 답변: 파리, 마르세유, 리옹 등이 있습니다.
추가 질문: 이 중 수도는 어디인가요?
중간 답변: 파리가 프랑스의 수도입니다.
최종 답변은: 파리
""",
    },
    {
        "question": "지구에서 가장 큰 바다는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 지구의 주요 대양에는 어떤 것들이 있나요?
중간 답변: 태평양, 대서양, 인도양 등이 있습니다.
추가 질문: 이 중 면적이 가장 넓은 바다는 무엇인가요?
중간 답변: 태평양이 면적이 가장 넓은 바다입니다.
최종 답변은: 태평양
""",
    },
    {
        "question": "세계에서 가장 긴 강은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 세계의 주요 강에는 어떤 것들이 있나요?
중간 답변: 나일강, 아마존강, 양쯔강 등이 있습니다.
추가 질문: 이 중 길이가 가장 긴 강은 무엇인가요?
중간 답변: 나일강이 가장 긴 강입니다.
최종 답변은: 나일강
""",
    },
    {
        "question": "태양계에서 가장 큰 행성은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 태양계의 행성들에는 어떤 것들이 있나요?
중간 답변: 수성, 금성, 지구, 화성, 목성, 토성, 천왕성, 해왕성 등이 있습니다.
추가 질문: 이 중 크기가 가장 큰 행성은 무엇인가요?
중간 답변: 목성이 가장 큰 행성입니다.
최종 답변은: 목성
""",
    },
    {
        "question": "인간의 심장은 하루에 몇 번 정도 박동하나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 일반적인 성인 심장의 분당 박동수는 어떻게 되나요?
중간 답변: 분당 60~100회 정도입니다.
추가 질문: 이를 바탕으로 하루 동안의 총 박동수를 계산해 줄 수 있나요?
중간 답변: 100,000회 정도입니다. (60회 * 60분 * 24시간을 기준으로)
최종 답변은: 약 10만 번
""",
    },
    {
        "question": "비행기를 처음으로 발명한 사람은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 초기 비행기 개발에 기여한 인물들은 누구인가요?
중간 답변: 라이트 형제, 오토 릴리엔탈 등이 있습니다.
추가 질문: 이 중 최초로 동력 비행에 성공한 사람은 누구인가요?
중간 답변: 라이트 형제(윌버 라이트와 오빌 라이트)입니다.
최종 답변은: 라이트 형제
""",
    },
    {
        "question": "모나리자를 그린 화가는 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 르네상스 시대의 주요 화가들은 누가 있나요?
중간 답변: 레오나르도 다빈치, 미켈란젤로, 라파엘로 등이 있습니다.
추가 질문: 이 중 '모나리자'라는 작품을 그린 사람은 누구인가요?
중간 답변: 레오나르도 다빈치가 그렸습니다.
최종 답변은: 레오나르도 다빈치
""",
    },
    {
        "question": "해리포터 시리즈의 작가는 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 영국 판타지 소설 작가 중 유명한 사람들은 누가 있나요?
중간 답변: J.R.R. 톨킨, J.K. 롤링 등이 있습니다.
추가 질문: 이 중 해리포터 시리즈를 쓴 사람은 누구인가요?
중간 답변: J.K. 롤링입니다.
최종 답변은: J.K. 롤링
""",
    },
    {
        "question": "BTS의 소속사는 어디인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: BTS가 속한 하이브 레이블에는 어떤 그룹들이 있나요?
중간 답변: 방탄소년단(BTS), 투모로우바이투게더(TXT), 르세라핌(LE SSERAFIM) 등이 있습니다.
추가 질문: 이들의 소속 회사 이름은 무엇인가요?
중간 답변: 빅히트 뮤직입니다.
최종 답변은: 빅히트 뮤직
""",
    },
    {
        "question": "손흥민 선수의 현재 소속팀은 어디인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 손흥민 선수가 활약하는 축구 리그는 어디인가요?
중간 답변: 잉글랜드 프리미어리그(EPL)입니다.
추가 질문: 이 리그에서 손흥민 선수가 뛰고 있는 팀은 어디인가요?
중간 답변: 토트넘 홋스퍼입니다.
최종 답변은: 토트넘 홋스퍼
""",
    },
    {
        "question": "김연아 선수의 주 종목은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 김연아 선수는 어떤 운동선수였나요?
중간 답변: 피겨스케이팅 선수였습니다.
추가 질문: 피겨스케이팅의 주 종목 중 김연아 선수가 가장 잘했던 종목은 무엇인가요?
중간 답변: 싱글 스케이팅입니다.
최종 답변은: 피겨스케이팅
""",
    },
    {
        "question": "골프에서 공을 치는 클럽의 종류는 몇 가지인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 골프 클럽의 주요 종류에는 어떤 것들이 있나요?
중간 답변: 우드, 아이언, 웨지, 퍼터 등이 있습니다.
추가 질문: 공식 경기에서 골퍼 한 명이 사용할 수 있는 클럽의 최대 개수는 몇 개인가요?
중간 답변: 최대 14개입니다.
최종 답변은: 14개
""",
    },
    {
        "question": "물은 몇 도에서 끓나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 순수한 물의 끓는점은 얼마인가요?
중간 답변: 100℃(섭씨 100도)입니다.
추가 질문: 이 온도는 어떤 기압 조건에서 측정되나요?
중간 답변: 표준 대기압(1기압) 조건에서입니다.
최종 답변은: 100℃
""",
    },
    {
        "question": "김치찌개에 들어가는 주재료는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 김치찌개에 반드시 들어가야 하는 재료는 무엇인가요?
중간 답변: 김치입니다.
추가 질문: 이 외에 일반적으로 사용되는 고기나 재료는 어떤 것들이 있나요?
중간 답변: 돼지고기, 두부, 양파, 파 등이 들어갑니다.
최종 답변은: 김치
""",
    },
    {
        "question": "자동차의 바퀴는 보통 몇 개인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 일반적인 승용차의 바퀴 수는 몇 개인가요?
중간 답변: 4개입니다.
추가 질문: 만약 예비용 타이어가 포함된다면 총 몇 개인가요?
중간 답변: 총 5개입니다.
최종 답변은: 4개
""",
    },
    {
        "question": "HTML에서 이미지 태그는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: HTML에서 이미지를 삽입할 때 사용되는 태그 이름은 무엇인가요?
중간 답변: <img> 태그입니다.
추가 질문: 이 태그의 필수 속성은 무엇인가요?
중간 답변: 이미지 소스를 지정하는 src 속성입니다.
최종 답변은: <img>
""",
    },
    {
        "question": "CSS에서 글자색을 바꾸는 속성은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: CSS에서 텍스트와 관련된 속성에는 어떤 것들이 있나요?
중간 답변: color, font-size, font-family 등이 있습니다.
추가 질문: 이 중 글자의 색상을 변경하는 속성은 무엇인가요?
중간 답변: color 속성입니다.
최종 답변은: color
""",
    },
    {
        "question": "자바스크립트에서 변수를 선언하는 키워드는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: ES6 이후에 도입된 변수 선언 키워드에는 어떤 것들이 있나요?
중간 답변: `let`과 `const`가 있습니다.
추가 질문: 이 중 재할당이 가능한 변수 선언 키워드는 무엇인가요?
중간 답변: `let`입니다.
최종 답변은: let
""",
    },
    {
        "question": "마케팅 4P에서 '가격'을 뜻하는 단어는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 마케팅 4P의 네 가지 요소는 무엇인가요?
중간 답변: Product(제품), Price(가격), Place(유통), Promotion(촉진)입니다.
추가 질문: 이 중 '가격'을 의미하는 단어는 무엇인가요?
중간 답변: Price입니다.
최종 답변은: Price
""",
    },
    {
        "question": "재무제표의 세 가지 주요 보고서는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 기업의 재무 상태를 나타내는 주요 보고서들에는 어떤 것들이 있나요?
중간 답변: 손익계산서, 재무상태표, 현금흐름표가 있습니다.
추가 질문: 이 중 가장 핵심적인 세 가지 보고서는 무엇인가요?
중간 답변: 손익계산서, 재무상태표, 현금흐름표입니다.
최종 답변은: 손익계산서, 재무상태표, 현금흐름표
""",
    },
]



In [24]:
from langchain_core.example_selectors import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,  # 의미적 유사성 예시 선택기
)
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.vectorstores import FAISS

# 랜덤 함수
import random
# Vector DB 생성 (고유한 저장소 이름 사용)
import time

collection_name = f"example_selector_{int(time.time())}"
chroma = Chroma(collection_name, OpenAIEmbeddings())

# 의미적 유사성을 측정하기 위해 OpenAIEmbeddings을 사용하여 단어를 숫자로 변환
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 여기에는 선택 가능한 예시 목록이 있습니다.
    examples,
    # 여기에는 의미적 유사성을 측정하는 데 사용되는 임베딩을 생성하는 임베딩 클래스가 있습니다.
    OpenAIEmbeddings(),  # 텍스트를 벡터로 변환하는 역할
    # 여기에는 임베딩을 저장하고 유사성 검색을 수행하는 데 사용되는 VectorStore 클래스가 있습니다.
    # Chroma,
    FAISS,
    # 이것은 생성할 예시의 수입니다.
    k=4,
)

# test_questions 목록에서 무작위로 질문을 선택
test_questions = [
    {"question": "행복의 의미는 무엇인가요?"},
    {"question": "시간을 효과적으로 관리하는 방법은 무엇인가요?"},
    {"question": "지속가능한 발전을 위해 우리가 할 수 있는 일은 무엇인가요?"},
    {"question": "현대 사회에서 인공지능이 가져올 가장 큰 변화는 무엇인가요?"},
    {"question": "좋은 리더의 자질에는 어떤 것들이 있나요?"},
]

# 1. 무작위 질문 선택
selected_question_dict = random.choice(test_questions)
question = selected_question_dict["question"]
# 입력과 가장 유사한 예시를 선택합니다.
selected_examples = example_selector.select_examples({"question": question})

print(f"입력에 가장 유사한 예시:\n{question}\n")
for example in selected_examples:
    print(f'question:\n{example["question"]}')
    print(f'answer:\n{example["answer"]}')

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


입력에 가장 유사한 예시:
현대 사회에서 인공지능이 가져올 가장 큰 변화는 무엇인가요?

question:
행렬 곱셈은 어떤 경우에 주로 사용되나요?
answer:
이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 대표적인 활용 분야는?
  중간 답변: 컴퓨터 그래픽스의 좌표변환, 머신러닝의 선형변환 등입니다.
  추가 질문: 가장 핵심적인 요점은?
  중간 답변: 선형변환을 조합하고 표현하는 데 사용됩니다.
  최종 답변은: 선형변환/좌표변환
  
question:
지구에서 가장 큰 바다는 무엇인가요?
answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 지구의 주요 대양에는 어떤 것들이 있나요?
중간 답변: 태평양, 대서양, 인도양 등이 있습니다.
추가 질문: 이 중 면적이 가장 넓은 바다는 무엇인가요?
중간 답변: 태평양이 면적이 가장 넓은 바다입니다.
최종 답변은: 태평양

question:
GPU 연산 플랫폼 CUDA를 만든 회사는 어디인가요?
answer:
이 질문에 추가 질문이 필요한가요: 아니오.
  최종 답변은: NVIDIA
  
question:
클라우드 서비스 모델 중 Gmail은 어떤 분류에 해당하나요?
answer:
이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 클라우드 서비스 모델의 종류는?
  중간 답변: IaaS, PaaS, SaaS가 있습니다.
  추가 질문: 완성된 애플리케이션을 사용하는 모델은?
  중간 답변: SaaS입니다.
  최종 답변은: SaaS
  


In [25]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 1) 예시가 출력될 포맷
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Q: {question}\nA:\n{answer}\n"
)

# 2) 선택기 기반 FewShot 프롬프트
fewshot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,   # 이미 위에서 만든 selector 그대로 사용
    example_prompt=example_prompt,
    prefix="=== 예시 시작 ===\n",
    suffix="=== 예시 끝 ===\n사용자 질문: {question}\n최종 답변:",
    input_variables=["question"],
)

# 3) LLM 체인
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
chain = fewshot_prompt | llm | StrOutputParser()

# 4) 실행 (위에서 정의한 question 그대로 사용 가능)
answer = chain.invoke({"question": question})
print("\n[모델 최종 응답]\n", answer)


[모델 최종 응답]
 이 질문에 추가 질문이 필요한가요: 예.  
추가 질문: 인공지능의 주요 활용 분야는?  
중간 답변: 의료, 금융, 제조업, 자율주행차 등 다양한 분야에서 활용됩니다.  
추가 질문: 이러한 변화가 사회에 미치는 영향은?  
중간 답변: 생산성 향상, 일자리 변화, 윤리적 문제 등이 있습니다.  
최종 답변은: 생산성 향상과 일자리 변화
